In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [8]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band6
  atmpro = mls
  band = 6
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1215
  vmin = 1100
  w_diffuse = [(1.66, 1.66, 1.7, 1.8)]
  wgt = [(0.3, 0.45, 0.6, 0.95)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312      1.0 -7.515858e-06                  NaN       NaN
0.000312      NaN           NaN                  1.0  0.000059
0.000750      2.0 -5.710839e-06                  2.0  0.000182
0.001052      3.0 -1.357627e-06                  3.0  0.000018
0.001476      4.0  4.142951e-06                  4.0  0.000012
0.002070      5.0  1.092976e-05                  5.0  0.000009
0.002904      6.0  1.921399e-05                  6.0  0.000008
0.004074      7.0  2.922501e-05                  7.0  0.000008
0.005714      8.0  4.123467e-05                  8.0  0.000009
0.008015      9.0  5.554334e-05                  9.0  0.000012
0.011243     10.0  7.248909e-05                 10.0  0.000015
0.015771     11.0  9.245346e-05                 11.0  0.000019
0.022122     12.0  1.158478e-04                 12.0  0.000024
0.031031     13.0  1.430235e-04                 13.0  0.000028
0.043528     14.0  1.706832e-04                 14.0  0.000034
0.061057     15.0  1.946068e-04                 15.0  0.000042
0.085645     16.0  2.172514e-04                 16.0  0.000049
0.120136     17.0  2.417442e-04                 17.0  0.000055
0.168516     18.0  2.683940e-04                 18.0  0.000062
0.236378     19.0  2.972043e-04                 19.0  0.000069
0.331549     20.0  3.282347e-04                 20.0  0.000077
0.465100     21.0  3.580843e-04                 21.0  0.000083
0.652400     22.0  3.340073e-04                 22.0  0.000078
0.915100     23.0  2.491289e-04                 23.0  0.000058
1.283650     24.0  1.683928e-04                 24.0  0.000037
1.800600     25.0  1.089072e-04                 25.0  0.000022
2.525700     26.0  6.578057e-05                 26.0  0.000011
3.542800     27.0  3.531527e-05                 27.0  0.000003
4.969550     28.0  1.664418e-05                 28.0 -0.000002
6.970850     29.0  6.625056e-06                 29.0 -0.000004
9.778100     30.0  3.057422e-07                 30.0 -0.000006
13.715850    31.0 -4.466561e-06                 31.0 -0.000008
19.239350    32.0 -7.917434e-06                 32.0 -0.000009
26.987250    33.0 -9.095245e-06                 33.0 -0.000009
37.855300    34.0 -8.292995e-06                 34.0 -0.000009
53.100050    35.0 -7.043769e-06                 35.0 -0.000009
73.887500    36.0 -5.842187e-06                 36.0 -0.000010
97.662500    37.0 -5.112114e-06                 37.0 -0.000010
121.437500   38.0 -5.476312e-06                 38.0 -0.000010
145.212500   39.0 -8.941120e-06                 39.0 -0.000015
168.987500   40.0 -1.368351e-05                 40.0 -0.000020
192.762500   41.0 -2.014969e-05                 41.0 -0.000027
216.537500   42.0 -2.686264e-05                 42.0 -0.000033
240.312500   43.0 -3.459951e-05                 43.0 -0.000040
264.087500   44.0 -4.428045e-05                 44.0 -0.000049
287.862500   45.0 -5.513548e-05                 45.0 -0.000058
311.637500   46.0 -5.870089e-05                 46.0 -0.000063
335.412500   47.0 -4.329596e-05                 47.0 -0.000058
359.187500   48.0 -5.453565e-06                 48.0 -0.000038
382.962500   49.0  6.869391e-05                 49.0 -0.000004
406.737500   50.0  2.318023e-04                 50.0  0.000087
430.512500   51.0  5.206409e-04                 51.0  0.000279
454.287500   52.0  9.647159e-04                 52.0  0.000614
478.062500   53.0  1.579269e-03                 53.0  0.001130
501.837500   54.0  2.365945e-03                 54.0  0.001851
525.612500   55.0  3.314556e-03                 55.0  0.002762
549.387500   56.0  4.403591e-03                 56.0  0.003845
573.162500   57.0  5.608275e-03                 57.0  0.005054
596.937500   58.0  6.894007e-03                 58.0  0.006324
620.712500   59.0  8.223797e-03                 59.0  0.007593
644.487500   60.0  9.557536e-03              

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -10.213940  0.000000e+00 -10.213940        NaN   
1.000000e-08 1            NaN           NaN        NaN -10.228444   
6.244000e-04 2     -10.213940  1.494929e-09 -10.213940 -10.228444   
8.759000e-04 3     -10.213940  2.148466e-09 -10.213940 -10.228444   
1.228600e-03 4     -10.213940  3.228538e-09 -10.213940 -10.228444   
1.723400e-03 5     -10.213940  5.014284e-09 -10.213940 -10.228444   
2.417400e-03 6     -10.213940  7.960200e-09 -10.213940 -10.228444   
3.390900e-03 7     -10.213940  1.281124e-08 -10.213940 -10.228444   
4.756500e-03 8     -10.213940  2.078292e-08 -10.213940 -10.228444   
6.672000e-03 9     -10.213940  3.385594e-08 -10.213940 -10.228444   
9.358900e-03 10    -10.213940  5.525289e-08 -10.213940 -10.228444   
1.312780e-02 11    -10.213940  9.020552e-08 -10.213940 -10.228444   
1.841450e-02 12    -10.213940  1.471930e-07 -10.213940 -10.228444   
2.583020e-02 13    -10.213940  2.399287e-07 -10.213940 -10.228444   
3.623230e-02 14    -10.213940  3.904559e-07 -10.213940 -10.228444   
5.082340e-02 15    -10.213940  6.303136e-07 -10.213940 -10.228444   
7.129060e-02 16    -10.213940  1.001164e-06 -10.213939 -10.228444   
1.000000e-01 17    -10.213940  1.566558e-06 -10.213938 -10.228444   
1.402710e-01 18    -10.213940  2.427698e-06 -10.213937 -10.228444   
1.967600e-01 19    -10.213939  3.738463e-06 -10.213935 -10.228444   
2.759970e-01 20    -10.213938  5.731156e-06 -10.213933 -10.228444   
3.871000e-01 21    -10.213937  8.754573e-06 -10.213928 -10.228443   
5.431000e-01 22    -10.213935  1.330311e-05 -10.213922 -10.228443   
7.617000e-01 23    -10.213932  1.930869e-05 -10.213913 -10.228443   
1.068500e+00 24    -10.213930  2.594905e-05 -10.213904 -10.228443   
1.498800e+00 25    -10.213928  3.282865e-05 -10.213895 -10.228443   
2.102400e+00 26    -10.213927  3.989121e-05 -10.213888 -10.228444   
2.949000e+00 27    -10.213928  4.706596e-05 -10.213881 -10.228445   
4.136600e+00 28    -10.213930  5.429652e-05 -10.213876 -10.228447   
5.802500e+00 29    -10.213935  6.187237e-05 -10.213873 -10.228450   
8.139200e+00 30    -10.213941  7.047554e-05 -10.213871 -10.228454   
1.141700e+01 31    -10.213951  8.070250e-05 -10.213871 -10.228460   
1.601470e+01 32    -10.213966  9.306118e-05 -10.213873 -10.228469   
2.246400e+01 33    -10.213988  1.083917e-04 -10.213879 -10.228482   
3.151050e+01 34    -10.214018  1.293355e-04 -10.213889 -10.228501   
4.420010e+01 35    -10.214062  1.609181e-04 -10.213901 -10.228529   
6.200000e+01 36    -10.214126  2.094299e-04 -10.213916 -10.228571   
8.577500e+01 37    -10.214213  2.800329e-04 -10.213933 -10.228633   
1.095500e+02 38    -10.214302  3.552011e-04 -10.213947 -10.228698   
1.333250e+02 39    -10.214395  4.328978e-04 -10.213963 -10.228768   
1.571000e+02 40    -10.214529  5.407418e-04 -10.213988 -10.228869   
1.808750e+02 41    -10.214709  6.825061e-04 -10.214026 -10.229008   
2.046500e+02 42    -10.214947  8.641227e-04 -10.214083 -10.229192   
2.284250e+02 43    -10.215236  1.077465e-03 -10.214159 -10.229417   
2.522000e+02 44    -10.215579  1.322551e-03 -10.214256 -10.229688   
2.759750e+02 45    -10.215986  1.605176e-03 -10.214381 -10.230011   
2.997500e+02 46    -10.216470  1.934219e-03 -10.214536 -10.230396   
3.235250e+02 47    -10.217045  2.343972e-03 -10.214701 -10.230855   
3.473000e+02 48    -10.217726  2.902962e-03 -10.214823 -10.231401   
3.710750e+02 49    -10.218524  3.684904e-03 -10.214839 -10.232046   
3.948500e+02 50    -10.219687  5.041800e-03 -10.214645 -10.233001   
4.186250e+02 51    -10.221460  7.467985e-03 -10.213992 -10.234473   
4.424000e+02 52    -10.223989  1.146310e-02 -10.212526 -10.236587   
4.661750e+02 53    -10.227394  1.758479e-02 -10.209809 -10.239455   
4.899500e+02 54    -10.231748  2.638652e-02 -10.205361 -10.243147   
5.137250e+02 55    -10.2

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')